In [1]:
import numpy as np
from numpy.random import default_rng
import transformers
import pandas as pd
from tqdm.notebook import tqdm
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
from recommender import Recommender
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import umap
import hdbscan
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from pylatexenc.latex2text import LatexNodes2Text

In [2]:
df = pd.read_csv('best_ru.csv')
df.dropna(inplace=True)
df.head()

,Unnamed: 0.1,Unnamed: 0,data_clean,data_unclean,url,topics
0,0,0,изложить метод проектирование устройство подач...,Изложен метод проектирования устройств подачи ...,https://cyberleninka.ru/article/n/matematiches...,-1
1,1,1,статья исследовательский метод оценка панель л...,В статье представлены исследовательские методы...,https://cyberleninka.ru/article/n/inzhenernaya...,-1
2,3,3,исследование актуальный философский проблема р...,Представлено исследование актуальной философск...,https://cyberleninka.ru/article/n/metodologich...,18
3,4,4,статья анализироваться суть понятие определить...,В статье анализируется суть как и понятия. О...,https://cyberleninka.ru/article/n/sotsiokultur...,0
4,5,5,рассматриваться создать сибирский отделение ра...,Рассматривается созданная в Сибирском отделени...,https://cyberleninka.ru/article/n/evolyutsiya-...,-1


In [4]:
torch.manual_seed(42)
np.random.seed(42)
torch.manual_seed(42)
transformers.set_seed(42)
tqdm.pandas()

# ADD additional data

In [5]:
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
sentence_model = SentenceTransformer(model_name, device="cuda")

In [6]:
embeddings = np.array([sentence_model.encode(x) for x in tqdm(df['data_clean'].tolist())])
np.save('embeddings_for_rudata.npy', embeddings)

  0%|          | 0/7577 [00:00<?, ?it/s]

In [7]:
rsys = Recommender(model_name, df['data_unclean'].tolist(), df['topics'].tolist(),
                   embeddings_file='embeddings_for_rudata.npy',
                   model_path='best_ru_model', additional_data=df['url'].tolist())

In [8]:
result = rsys.recommend(['программирование', 'аналитика', 'собеседование', 'стажер'])
result

Getting recommendations:   0%|          | 0/4 [00:00<?, ?it/s]

{'программирование': (['Программирование — это не только когда пишешь код, запускаешь его и удовлетворенно наблюдаешь за его безупречной работой, ведь зачастую он работает совсем не так, как мы рассчитываем! Возникает...',
   'Парсером называется часть программы, которая из линейной последовательности простых данных строит более сложные структуры данных с учетом некоторой грамматики. \r\n\r\nФункциональные языки...',
   'Час кода — это глобальная инициатива, часовое введение в программирование прямо на школьном уроке, призванное развеять мифы о сложности кодинга и показать, что каждый может научиться его основам. В...'],
  array(['https://habrahabr.ru/company/pvs-studio/blog/318658/',
         'https://habrahabr.ru/post/317304/',
         'https://habrahabr.ru/post/314950/'], dtype='<U173')),
 'аналитика': (['Рассмотрен широкий класс новых методов , основанный на статистической оценке инвариантов группы автоморфизмов изображения. Разработан теоретико-групповой статистический подход к р

In [11]:
result = rsys.recommend(['программирование', 'аналитика', '', 'стажер'])
result

Getting recommendations:   0%|          | 0/4 [00:00<?, ?it/s]

{'программирование': (['Программирование — это не только когда пишешь код, запускаешь его и удовлетворенно наблюдаешь за его безупречной работой, ведь зачастую он работает совсем не так, как мы рассчитываем! Возникает...',
   'Парсером называется часть программы, которая из линейной последовательности простых данных строит более сложные структуры данных с учетом некоторой грамматики. \r\n\r\nФункциональные языки...',
   'Час кода — это глобальная инициатива, часовое введение в программирование прямо на школьном уроке, призванное развеять мифы о сложности кодинга и показать, что каждый может научиться его основам. В...'],
  array(['https://habrahabr.ru/company/pvs-studio/blog/318658/',
         'https://habrahabr.ru/post/317304/',
         'https://habrahabr.ru/post/314950/'], dtype='<U173')),
 'аналитика': (['Рассмотрен широкий класс новых методов , основанный на статистической оценке инвариантов группы автоморфизмов изображения. Разработан теоретико-групповой статистический подход к р

In [15]:
result = rsys.recommend(['трансформеры'])
result

Getting recommendations:   0%|          | 0/1 [00:00<?, ?it/s]

{'трансформеры': (['Представлены результаты анализа систем электроснабжения для  с полностью электрифицированным оборудованием. Сформулированы требования к современным . Сделаны выводы о предпочтительном использовании систем генерирования энергии на основе  в сочетании с многоуровневыми активными выпрямителями и инверторами на MOSFET-модулях.',
   '\r\n\r\nДвигатель постоянного тока, стоящий в моноколесе, может работать как генератор, заряжая батареи на торможении. Однако возможность еще не означает, что рекуперация непременно есть. Из того, что...',
   'Рассматривается численно-аналитическая модель инвертирующего преобразователя, работающего как в режиме непрерывных токов индуктивного накопителя энергии, так и в режиме прерывистых токов. Приведены результаты расчетов бифуркационных диаграмм.'],
  array(['https://cyberleninka.ru/article/n/perspektivnye-sistemy-elektrosnabzheniya-samoleta-s-polnostyu-elektrifitsirovannym-oborudovaniem',
         'https://geektimes.ru/post/282130/',
    